Imports and Setups

In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

from pathlib import Path
import time

In [3]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

Paths and configs

In [4]:
DATA_DIR = Path("../data/raw")
TRAIN_DIR = DATA_DIR / "training"
VAL_DIR = DATA_DIR / "validation"

BATCH_SIZE = 32
NUM_CLASSES = 11
IMAGE_SIZE = 224

Transforms

In [5]:

train_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )

])
val_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


DATASET AND DATALOADER

In [6]:
train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
val_dataset = datasets.ImageFolder(VAL_DIR, transform=val_transforms)

class_names = train_dataset.classes
print("Classes:", class_names)

Classes: ['Bread', 'Dairy product', 'Dessert', 'Egg', 'Fried food', 'Meat', 'Noodles-Pasta', 'Rice', 'Seafood', 'Soup', 'Vegetable-Fruit']


In [9]:

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=False
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=False
)

Loading Baseline Model

In [10]:
model = models.resnet18(pretrained=True)

# Replace classifier here changing 1000 class of resnet to our custom 11 clasess to train on
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, NUM_CLASSES)

model.load_state_dict(torch.load("best_model.pth", map_location=device))
model = model.to(device)

/Applications/anaconda3/envs/foodsense/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Applications/anaconda3/envs/foodsense/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fine-Tuning and unfreeze correctly

In [11]:
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

Loss and optimizer using two learning rates

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW([
    {"params": model.layer4.parameters(), "lr": 1e-4},
    {"params": model.fc.parameters(), "lr": 1e-3}
])

Training and Validation Loop

In [13]:
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total

def validate(model, loader, criterion):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / total, correct / total

Model Training Loop Finetuning

In [15]:
NUM_EPOCHS = 10
best_val_acc = 0.0

start_time = time.time()

for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer
    )

    val_loss, val_acc = validate(
        model, val_loader, criterion
    )

    print(
        f"Epoch [{epoch+1}/{NUM_EPOCHS}] | "
        f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
        f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
    )

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "finetuned_resnet18.pth")
        print("✅ Best model saved")

total_time = time.time() - start_time
print(f"Training completed in {total_time/60:.2f} minutes")

Epoch [1/10] | Train Loss: 0.2912 | Train Acc: 0.9030 | Val Loss: 0.4488 | Val Acc: 0.8601
✅ Best model saved
Epoch [2/10] | Train Loss: 0.1907 | Train Acc: 0.9391 | Val Loss: 0.4364 | Val Acc: 0.8650
✅ Best model saved
Epoch [3/10] | Train Loss: 0.1340 | Train Acc: 0.9541 | Val Loss: 0.4526 | Val Acc: 0.8694
✅ Best model saved
Epoch [4/10] | Train Loss: 0.1075 | Train Acc: 0.9653 | Val Loss: 0.4797 | Val Acc: 0.8761
✅ Best model saved
Epoch [5/10] | Train Loss: 0.0951 | Train Acc: 0.9650 | Val Loss: 0.5145 | Val Acc: 0.8746
Epoch [6/10] | Train Loss: 0.0947 | Train Acc: 0.9665 | Val Loss: 0.5224 | Val Acc: 0.8679


KeyboardInterrupt: 

In [16]:
print(f"Best Validation Accuracy: {best_val_acc:.4f}")


Best Validation Accuracy: 0.8761


In [8]:
import json

class_names = train_dataset.classes

with open("artifacts/class_mapping.json", "w") as f:
    json.dump(class_names, f)

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/class_mapping.json'